<a href="https://colab.research.google.com/github/matinmazid/York_ML_certificate/blob/master/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load all the dependant libraries

# Scenario 0 No context provided to model

In [1]:
!pip --quiet install langchain langchain_community pypdf langchain_core langchain-text-splitters langchain-chroma
!pip --quiet install chromadb

In [34]:
#apiKey=input("Enter your openai api key")
print("hidden")

hidden


In [4]:
import os
os.environ['OPENAI_API_KEY']=apiKey

Create the lang chain

In [5]:
from langchain.chat_models import ChatOpenAI
import langchain_core
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
model = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo")
output_parser = StrOutputParser()
chatPrompt=ChatPromptTemplate.from_messages(
    [("system","you are a financial specialist with experience in Aritifical Intelligence"),
     ("user","what is XBRL and its relationship to AICPA")]
    )

<ipython-input-5-749de0fa123d>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo")


In [6]:
chain= chatPrompt | model | output_parser

In [7]:
chain.invoke({})


'XBRL stands for eXtensible Business Reporting Language, which is a standardized format for the electronic communication of business and financial data. It allows companies to easily and accurately transmit financial information in a machine-readable format. XBRL tags data using a standardized set of codes, making it easier for investors, analysts, regulators, and other stakeholders to access and analyze financial information.\n\nThe American Institute of Certified Public Accountants (AICPA) is a professional organization for certified public accountants in the United States. The AICPA has been involved in promoting the use of XBRL for financial reporting. The AICPA has developed guidance and resources to help accountants and companies implement XBRL effectively in their financial reporting processes.\n\nIn summary, the AICPA has been supportive of the adoption of XBRL as a tool for improving the accuracy and efficiency of financial reporting, and has provided guidance to its members o

# Scenario 1 Load pdf document loader saving to Chromadb

In [8]:
!pip install --upgrade --quiet pypdf
!pip install --upgrade --quiet langchain_openai
!pip install tiktoken

https://www.stern.nyu.edu/sites/default/files/assets/documents/ChenChoDouLev2021WP.pdf

https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import tiktoken
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

loader = PyPDFLoader("ChenChoDouLev2021WP.pdf")
docs = loader.load()


# split and chuck the input data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
# text splitter knows how to extract the text of a sourced doc
all_splits = text_splitter.split_documents(docs)

# all_texts = [cleanStr(doc.page_content) for doc in all_splits]
# all_texts


vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("what is XBRL and its relationship to AICPA")




/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'XBRL is the eXtensible Business Reporting Language used for financial reporting. The AICPA is involved in developing data quality tools and working with the FASB on U.S. GAAP taxonomy enhancements related to XBRL. The Data Quality Committee of XBRL U.S. sets guidance and validation rules for XBRL reporting.'

# Scenario 2 Pine cone database and webcrawler
https://python.langchain.com/docs/integrations/document_loaders/recursive_url/
https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm


In [10]:
!pip install -qU langchain-community beautifulsoup4 lxml

In [11]:
import re
from bs4 import BeautifulSoup
# import html as htmlTool


def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    someText=re.sub(r"\n\n+", "\n\n", soup.text).strip()
    # someText=htmlTool.unescape(someText)
    # the test page had non-breaking space as well as other non acii characters
    # we get rid of them
    return someText.encode('latin-1', 'ignore').decode('ascii',"ignore")


In [33]:
from langchain_community.document_loaders import RecursiveUrlLoader
loader=RecursiveUrlLoader(url="https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm",
                           use_async=False
                          ,extractor=bs4_extractor)

docs_html = loader.load()

# text splitter knows how to extract the text of a sourced doc
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
# text splitter knows how to extract the text of a sourced doc
all_splits = text_splitter.split_documents(docs_html)






[Document(metadata={'source': 'https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm', 'content_type': 'text/html; charset=UTF-8', 'title': 'The Daily\xa0—\xa0Labour Force Survey, January 2025', 'description': 'Employment increased by 76,000 (+0.4%) in January and the employment rate rose 0.1 percentage points to 61.1%. The unemployment rate declined 0.1 percentage points to 6.6%.', 'language': 'en'}, page_content='The DailyLabour Force Survey, January 2025\n\nSkip to main content\n\nSkip to "About this site"\n\nLanguage selection\n\nFranais\n\n / Statistique Canada\n\nSearch and menus\n\nSearch and menus\n\nSearch\n\nSearch website\n\nSearch\n\nTopics menu\n\nSubjects\nData\nAnalysis\nReference\nGeography\nCensus\nSurveys and statistical programs\nAbout StatCan\nCanada.ca\n\nHome\nThe Daily\n\nThe Daily\n|\n\nSearch The Daily\n\nSearch website\n\nSearch\n\nIn the news\nIndicators\nReleases by subject\n\nSpecial interest\nRelease schedule\nInformation\n\nLabour Force

Pinecone requires a API key.
https://docs.pinecone.io/guides/get-started/quickstart

In [13]:
!pip install protobuf==3.20.*
!pip install  "pinecone[grpc]"
!pip install --upgrade --quiet  lark
!pip install --upgrade --quiet pinecone-notebooks pinecone-client


  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
protoc-gen-openapiv2 0.0.1 requires protobuf>=4.21.0, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.30.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [17]:
# "pcsk_6hdvi3_87n6EcsXGey2yAEkcDEme2W6Dw5jCakxvnH7SRjTLeNLMM7uze9KV1ADqgU8q13"

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

pc = Pinecone(api_key="")

In [18]:

embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d.page_content for d in docs_html],
    parameters={
        "input_type": "passage",
        "truncate": "END"
    }
)

print(embeddings)

EmbeddingsList(
  model='multilingual-e5-large',
  data=[
    {'values': [0.018951416015625, 0.0171661376953125, ..., -0.0333251953125, 0.01323699951171875]}
  ],
  usage={'total_tokens': 512}
)
